In [3]:
import tensorflow as tf

In [5]:
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Input,Conv2D,Dense,Flatten,Dropout, \
GlobalMaxPooling2D, MaxPooling2D, BatchNormalization # Changed 'GlobalMaxPooling2d' to 'GlobalMaxPooling2D'
from tensorflow.keras.models import Model

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='...')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [9]:
# Note : model creation must be in strategy scope
# We will define the function now, but tihs code
# won't run outside the scope

In [ ]:
# Build the model using the functional API
def create_model():
  i = Input(shape=x_train[0].shape)

  x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = BatchNormalization()(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)

  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(512, activation='softmax')(x)

  model = Model(i, x)
  return model

In [ ]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print("x_train.shape:", x_train.shape)
print("y_train.shape", y_train.shape)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [ ]:
# Critical part
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

batch_size = 256

# reshuffle_each_iteration = None is default but is later set to True if None
# thus "True" is the actual default
train_dataset = train_dataset.shuffle(1000).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [ ]:
# Fit
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=5)

In [ ]:
model.save('mymodel.h5')

In [ ]:
with strategy.scope():
  model = tf.keras.models.load_model('mymodel.h5')
  out = model.predict(x_test[:1])
  print(out)

In [ ]:
# Note old bug
# https://www.kaggle.com/c/flower-classification-withitpus/discussion/148615